## Algorithme HHL (Harrow-Hassidim-Lloyd)
L'algorithme HHL est un algorithme quantique qui permet de résoudre certains systèmes d'équations linéaires.

In [1]:
import cirq
import math
import numpy as np
import sympy
from cirq.contrib.svg import SVGCircuit

À partir d'une matrice Hermitienne A connue et d'un vecteur b connu, trouver le vecteur x tel que Ax=b.
L'algorithme utilise 3 ensembles de qubits disctincs. D'abord, un registre de qubits permettent l'estimation de phase pour extraire les valeurs propres de la matrice A, puis des qubits de mémoire pour enregistrer les valeurs des vecteurs b et x et finalement un qubit ancilla ( ou qubit auxiliaire) nécessaire aux calculs.

L'algorithme HHL transforme d'abord cette matrice A en opérateur unitaire sur lequel on fait une estimation de phase.

On définit une porte qui applique l'estimation de phase sur un nombre arbitraire de qubits. Le dernier qubit encode le vecteur propre tandis que les autres qubits encodent l'estimation de phase. La porte prends en paramètre le nombre de qubits ainsi que le qubit dont on veut estimer la phase

In [2]:
class EstimationPhase(cirq.Gate):

    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        yield cirq.H.on_each(*qubits[:-1])
        yield PhaseKickback(self.num_qubits(), self.U)(*qubits)
        yield cirq.qft(*qubits[:-1], without_reverse=True) ** -1

Comme l'estimation de phase que nous venons d'implémenter utilise la porte de retour de phase (phase kickback en anglais), nous devons aussi implémenter cette porte.

In [3]:
class PhaseKickback(cirq.Gate):

    def __init__(self, num_qubits, unitary):
        super(PhaseKickback, self)
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            yield cirq.ControlledGate(self.U ** (2**i))(qubit, memory)

On implémente aussi la porte de simulation Hamiltonienne qui transforme la matrice Hermitienne A en opérateur unitaire.

Pour les fins de l'exercice, on utilise la fonction de la librairie numpy, np.linalg.eigh(A), pour simplifier la simulation Hamiltonienne. Pour des plus grandes matrices, on devra implémenter une vrai simulation Hamiltonienne.

In [4]:
class SimulationHamiltonienne(cirq.EigenGate):

    def __init__(self, A, t, exponent=1.0):
        cirq.EigenGate.__init__(self, exponent=exponent)
        self.A = A
        self.t = t
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w * t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))

    def _num_qubits_(self) -> int:
        return 1

    def _with_exponent(self, exponent):
        return SimulationHamiltonienne(self.A, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components

L'algorithme HHL utilise des rotations controllées autour de l'axe Y du qubit ancilla. Ici, le qubit ancilla est le dernier qubit du circuit.

In [5]:
class RotationPropre(cirq.Gate):

    def __init__(self, num_qubits, C, t):
        super(RotationPropre, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2 ** (num_qubits - 1)

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        for k in range(self.N):
            kGate = self._ancilla_rotation(k)

            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k - 1)

            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    yield cirq.X(q)
                xor >>= 1

                # Build controlled ancilla rotation
                kGate = cirq.ControlledGate(kGate)

            yield kGate(*qubits)

    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2 * math.asin(self.C * self.N * self.t / (2 * math.pi * k))
        return cirq.ry(theta)


Assemblons maintenant le circuit. Les paramètre à donner sont A: la matrice hermitienne d'entrée, C: la plus petite valeur propre qui peut être représentée par le circuit, t: variable qui controle la précision de l'algorithme, register_size: la taille du registre et input_prep_gates des portes à ajouter au circuit avant l'algorithme HHL.

In [6]:
def hhl_circuit(A, C, t, register_size, *input_prep_gates):

    ancilla = cirq.LineQubit(0)
    # to store eigenvalues of the matrix
    register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # to store input and output vectors
    memory = cirq.LineQubit(register_size + 1)

    c = cirq.Circuit()
    hs = SimulationHamiltonienne(A, t)
    pe = EstimationPhase(register_size + 1, hs)
    c.append([gate(memory) for gate in input_prep_gates])
    c.append(
        [
            pe(*(register + [memory])),
            RotationPropre(register_size + 1, C, t)(*(register + [ancilla])),
            pe(*(register + [memory])) ** -1,
            cirq.measure(ancilla, key='a'),
        ]
    )

    c.append(
        [
            cirq.PhasedXPowGate(
                exponent=sympy.Symbol('exponent'), phase_exponent=sympy.Symbol('phase_exponent')
            )(memory),
            cirq.measure(memory, key='m'),
        ]
    )

    return c

Visualisons le circuit généré

In [7]:
# Eigendecomposition:
#   (4.537, [-0.971555, -0.0578339+0.229643j])
#   (0.349, [-0.236813, 0.237270-0.942137j])
# |b> = (0.64510-0.47848j, 0.35490-0.47848j)
# |x> = (-0.0662724-0.214548j, 0.784392-0.578192j)

A = np.array(
    [
        [4.30213466 - 6.01593490e-08j, 0.23531802 + 9.34386156e-01j],
        [0.23531882 - 9.34388383e-01j, 0.58386534 + 6.01593489e-08j],
    ]
)
t = 0.358166 * math.pi
register_size = 4
input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
expected = (0.144130, 0.413217, -0.899154)

# Set C to be the smallest eigenvalue that can be represented by the
# circuit.
C = 2 * math.pi / (2**register_size * t)

circuit=hhl_circuit(A, C, t, register_size, *input_prep_gates)
SVGCircuit(hhl_circuit(A, C, t, register_size, *input_prep_gates))

Définissons comment simuler le circuit.

In [8]:
def simulate(circuit):
    simulator = cirq.Simulator()

    # Cas pour mesurer X,Y et Z respectivement sur les qubits de mémoire,
    params = [
        {'exponent': 0.5, 'phase_exponent': -0.5},
        {'exponent': 0.5, 'phase_exponent': 0},
        {'exponent': 0, 'phase_exponent': 0},
    ]

    results = simulator.run_sweep(circuit, params, repetitions=5000)

    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        # Sélectionner uniquement les cas ou le qubit auxiliaire est 1.
        expectation = 1 - 2 * np.mean(result.measurements['m'][result.measurements['a'] == 1])
        print(f'{label} = {expectation}')

Lancer la simulation

In [9]:
A = np.array(
    [
        [4.30213466 - 6.01593490e-08j, 0.23531802 + 9.34386156e-01j],
        [0.23531882 - 9.34388383e-01j, 0.58386534 + 6.01593489e-08j],
    ]
)
t = 0.358166 * math.pi
register_size = 4
input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
expected = (0.144130, 0.413217, -0.899154)
C = 2 * math.pi / (2**register_size * t)

print("Résultats attendus :")
print("X =", expected[0])
print("Y =", expected[1])
print("Z =", expected[2])
print("Résultats obtenus : ")

simulate(hhl_circuit(A, C, t, register_size, *input_prep_gates))

Résultats attendus :
X = 0.14413
Y = 0.413217
Z = -0.899154
Résultats obtenus : 
X = 0.12567463377023902
Y = 0.4107692307692308
Z = -0.8999999999999999


L'implémentation ci-haut est adaptée à partir de l'implémentation de la librairie Cirq. Plus précisément de [cet exemple](https://github.com/quantumlib/Cirq/blob/master/examples/hhl.py)

Cirq Developers. (2022). Cirq. Zenodo. https://doi.org/10.5281/zenodo.7465577, 